In [1]:
import SpectralConvxD as spc



In [2]:
x_train, y_train,x_test, y_test=spc.generate_data(name_data='mnist1d')


Did or could not load data from ./mnist1d_data.pkl. Rebuilding dataset...


In [3]:
models_pars={'reference':
                    {"spectral_config":{ 'is_base_trainable': True,
                                        'is_diag_start_trainable': False,
                                        'is_diag_end_trainable': False,
                                        'use_bias': True
                                        },
                     "spectral_cnn1d_config":{ 'kernel_size': 3,
                                                'stride': 1,
                                                'padding': 0,
                                                'trainable_phi':True,
                                                'use_lambda_out':False,
                                                'use_lambda_in' : False,
                                                'use_bias': True
                                              },
                     "spectral_cnn2d_config":{ 'kernel_size': 3,
                                                'strides': 1,
                                                'padding': 'VALID',
                                                "use_lambda_out":False,
                                                "use_lambda_in":False,
                                                "use_encode":False,
                                                "use_decode":False,
                                                'use_bias': True,
                                                "trainable_omega_diag":True,
                                                "trainable_omega_triu":True,
                                                "trainable_omega_tril":True,
                                                "trainable_aggregate":False,
                                                }
                    },
                    
             'Dspec':{"spectral_config":{ 'is_base_trainable': True,
                                        'is_diag_start_trainable': False,
                                        'is_diag_end_trainable': False,
                                        'use_bias': True
                                        },
                     "spectral_cnn1d_config":{ 'kernel_size': 3,
                                                'stride': 1,
                                                'padding': 0,
                                                'trainable_phi':False,
                                                'use_lambda_out':False,
                                                'use_lambda_in' : True,
                                                'use_bias': True
                                              },
                     "spectral_cnn2d_config":{ 'kernel_size': 3,
                                                'strides': 1,
                                                'padding': 'VALID',
                                                "use_lambda_out":False,
                                                "use_lambda_in":False,
                                                "use_encode":False,
                                                "use_decode":True,
                                                'use_bias': True,
                                                "trainable_omega_diag":False,
                                                "trainable_omega_triu":False,
                                                "trainable_omega_tril":False,
                                                "trainable_aggregate":False,
                                                }
                    },
             
             'specConvXd':{"spectral_config":{ 'is_base_trainable': False,
                                        'is_diag_start_trainable': False,
                                        'is_diag_end_trainable': True,
                                        'use_bias': True
                                        },
                     "spectral_cnn1d_config":{ 'kernel_size': 3,
                                                'stride': 1,
                                                'padding': 1,
                                                'trainable_phi':True,
                                                'use_lambda_out':False,
                                                'use_lambda_in' : True,
                                                'use_bias': True
                                              },
                     "spectral_cnn2d_config":{ 'kernel_size': 3,
                                                'strides': 1,
                                                'padding': 'VALID',
                                                "use_lambda_out":False,
                                                "use_lambda_in":False,
                                                "use_encode":False,
                                                "use_decode":True,
                                                'use_bias': True,
                                                "trainable_omega_diag":False,
                                                "trainable_omega_triu":False,
                                                "trainable_omega_tril":False,
                                                "trainable_aggregate":False,
                                                }
                    }
            }
models_name=list(models_pars.keys())
print(models_name)

['reference', 'Dspec', 'specConvXd']


In [4]:
maxpooling_config ={'strides': 1,
                     'padding': 'valid' }

hyperparameters = { 'filters' :20,
                    'input_shape' :(40,),
                    'learning_rate' : 0.01,
                    'epochs' : 20,
                    'batch_size' :100,
                    'activation': 'relu',
                    'labels' : 10,
                    'conxd':1,
                    'pool_size':2,
                    'pruning':None,
                    'Use_pruning':None,
                    'use_base_and_Lambda':None}

In [5]:
models = spc.SpectralCnn(hyperparameters=hyperparameters,maxpooling_config=maxpooling_config)

In [6]:
inv=2
models.compile_models(units=2500,spectral_config=models_pars.get(models_name[inv]).get('spectral_config'),
                      spectral_cnn1d_config=models_pars.get(models_name[inv]).get('spectral_cnn1d_config'),
                      spectral_cnn2d_config=models_pars.get(models_name[inv]).get('spectral_cnn2d_config'),
                      name=models_name[inv])
models.summary(name=models_name[inv])


specConvXd model summary:
 


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spec_cnn1d (SpecCnn1D)          │ (None, 40, 20)         │           920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 39, 20)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 780)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral (Spectral)             │ (None, 2500)           │         5,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_1 (Spectral)           │ (None, 10)             │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,940 (23.20 KB)

 Trainable params: 5,940 (23.20 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
models.train(x_train, y_train, x_test, y_test, name=models_name[inv],verbose=1,layers=(3,4))

Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2331 - loss: 4.9475 - val_accuracy: 0.4550 - val_loss: 1.4280
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4935 - loss: 1.3364 - val_accuracy: 0.5310 - val_loss: 1.2364
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5713 - loss: 1.1431 - val_accuracy: 0.5870 - val_loss: 1.0838
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6306 - loss: 0.9941 - val_accuracy: 0.6115 - val_loss: 1.0178
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6537 - loss: 0.9270 - val_accuracy: 0.6445 - val_loss: 0.9557
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6899 - loss: 0.8252 - val_accuracy: 0.6680 - val_loss: 0.8624
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6912 - loss: 0.8005 - val_accuracy: 0.6750 - val_loss: 0.8409
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7248 - loss: 0.7400 - val_accuracy: 0.6985 - val_loss:

In [8]:
models.get_hyperparameters()

{'filters': 20,
 'input_shape': (40,),
 'learning_rate': 0.01,
 'epochs': 20,
 'batch_size': 100,
 'activation': 'relu',
 'labels': 10,
 'conxd': 1,
 'pool_size': 2,
 'pruning': None,
 'Use_pruning': None,
 'use_base_and_Lambda': None,
 'units': 2500}